<a href="https://colab.research.google.com/github/skssushil/Web-mining/blob/master/1_6_Apriori_Algorithm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import nltk
from nltk.util import ngrams
import numpy as np
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
from collections import Counter
import pandas as pd
import re
nltk.download('punkt');
nltk.download('stopwords');

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
# read in data
corpus = []
dataSetFilename = 'text1.txt'
with open(dataSetFilename,'r') as file:
  corpus=file.read()

In [ ]:
N=len(corpus)

In [ ]:
N

27715

##Preprocessing 

In [ ]:
def convert_lower_case(data):
  return np.char.lower(data)

In [ ]:
def remove_stop_words(data):
  stop_words = stopwords.words('english')
  words = word_tokenize(str(data))
  new_text = ""
  for w in words:
    if w not in stop_words and len(w) > 1:
      new_text = new_text + " " + w
  return new_text

In [ ]:
def remove_punctuation(data):
  symbols = "!\"#$%&()*+-./:;<=>?@[\]^_`{|}~\n"
  for i in range(len(symbols)):
    data = np.char.replace(data, symbols[i], ' ')
    data = np.char.replace(data, "  ", " ")
    data = np.char.replace(data, ',', '')
  return data

In [ ]:
def remove_apostrophe(data):
  return np.char.replace(data, "'", "")

In [ ]:
def stemming(data):
  stemmer= PorterStemmer()  
  tokens = word_tokenize(str(data))
  new_text = ""
  for w in tokens:
    new_text = new_text + " " + stemmer.stem(w)
  return new_text

In [ ]:
import inflect 
p = inflect.engine()   
# convert number into words 
def convert_number(text): 
  temp_str = text.split() 
  new_string = []  
  for word in temp_str:
    if word.isdigit(): 
      temp = p.number_to_words(word) 
      new_string.append(temp) 
    else:
      new_string.append(word) 
      temp_str = ' '.join(new_string) 
  return temp_str

In [ ]:
def preprocess(data):
  data = convert_number(data)
  data = convert_lower_case(data)
  data = remove_punctuation(data) #remove comma seperately
  data = remove_apostrophe(data)
  data = remove_stop_words(data)
  data = stemming(data)
  data = remove_punctuation(data)
  data = stemming(data) #needed again as we need to stem the words
  data = remove_punctuation(data) #needed again as num2word is giving few hypens and commas fourty-one
  data = remove_stop_words(data) #needed again as num2word is giving stop words 101 - one hundred and one
  data = convert_number(data)
  return data

In [ ]:
tokens=word_tokenize(preprocess(corpus))

In [23]:
print(" Vocabs Length ""\n",len(tokens))

 Vocabs Length 
 2381


In [25]:
print(" Vocabs ""\n",*tokens[:500])

 Vocabs 
 emma woodhou handsom clever rich comfort home happi disposit seem unit best bless exist live nearli twenti one year world littl distress vex youngest two daughter affect indulg father consequ sister marriag mistress hou earli period mother die long ago indistinct remembr caress place suppli excel woman gover fallen littl short mother affect sixteen year miss taylor mr woodhou famili less gover friend fond daughter particularli emma intimaci sister even miss taylor cea hold nomin offic gover mild temper hardli allow impo restraint shadow author long pass away live togeth friend friend mutual attach emma like highli esteem miss taylor judgment direct chiefli real evil ind emma situat power rather much way disposit think littl well disadvantag threaten alloy mani enjoy danger howev present unperceiv mean rank misfortun sorrow came—a gentl sorrow—but shape disagr consciou —miss taylor marri miss taylor loss first brought grief wed day belov friend emma first sat mourn thought con

##Apriori

In [ ]:
def createCandidateSet(data):
	cand = []
	for row in data:
		for itm in row:
			if [itm] not in cand:
				cand.append([itm])
	cand.sort()
	return list(map(frozenset,cand))

In [ ]:
def scanData(data, candidateSet, minSupport):
	subsetCount = {}
	for curSet in data:
		for cand in candidateSet:
			if cand.issubset(curSet):
				if not cand in subsetCount:
					subsetCount[cand] = 1
				else:
					subsetCount[cand] += 1
	n = float(len(data))
	valid = []
	for key in subsetCount:
		sup = subsetCount[key]
		if sup >= minSupport:
			valid.insert(0,key)
	return valid, subsetCount

In [ ]:
def genApriori(freqSets, k):
	valid = []
	nFreqSets = len(freqSets)
	for i in range(nFreqSets):
		for j in range(i+1, nFreqSets):
			lstCands1 = list(freqSets[i])[:k-2]
			lstCands2 = list(freqSets[j])[:k-2]
			lstCands1.sort()
			lstCands2.sort()
			# if first k-2 elements are equal
			if lstCands1 == lstCands2:
				valid.append(freqSets[i]|freqSets[j]) # union 
	return valid

In [ ]:
def apriori(data, minSupport):
	candSet = createCandidateSet(data)
	setData = list(map(set,data))
	lstCands, subsetCounts = scanData(setData,candSet,minSupport)
	lstCands = [lstCands]
	k = 2
	while(len(lstCands[k-2]) > 0):
		candSetX = genApriori(lstCands[k-2],k)
		lstCandsX, subsetCountsX = scanData(setData,candSetX, minSupport)
		subsetCounts.update(subsetCountsX)
		lstCands.append(candSetX)
		k += 1
	return lstCands, subsetCounts

In [ ]:
# read in data
data = []
dataSetFilename = 'text.txt'
with open(dataSetFilename,'r') as file:
	for line in file:
		data.append(line.strip().split(','))

In [ ]:
print("What min. support do you want to use? ")
minSupp = input()
minSupp = int(minSupp)

What min. support do you want to use? 
2


In [ ]:
print("\n**** Apriori with minSupport = {} ****".format(minSupp))
# call apriori
sets, counts = apriori(data,minSupp)
print("\nSets:")
"""for x in sets:
	for y in x:
		print(y)"""
print("\n Most Frequent items of size upto 5: \n")
for k,v in sorted(counts.items(), reverse=True, key=lambda tup: tup[1])[:20]:
	print(k, v)


**** Apriori with minSupport = 2 ****

Sets:

 Most Frequent items of size upto 5: 

frozenset({'BabyRuth'}) 49
frozenset({'Snickers'}) 47
frozenset({'Hershey'}) 43
frozenset({'AlmondJoy'}) 41
frozenset({'KitKat'}) 40
frozenset({'Snickers', 'Hershey'}) 40
frozenset({'BabyRuth', 'Snickers'}) 40
frozenset({'BabyRuth', 'Hershey'}) 39
frozenset({'BabyRuth', 'Snickers', 'Hershey'}) 36
frozenset({'HeathBar'}) 35
frozenset({'Twix'}) 35
frozenset({'MilkyWay'}) 34
frozenset({'Snickers', 'KitKat'}) 34
frozenset({'Cotton Candy'}) 33
frozenset({'BabyRuth', 'KitKat'}) 33
frozenset({'Cadbury'}) 32
frozenset({'Snickers', 'AlmondJoy'}) 32
frozenset({'BabyRuth', 'AlmondJoy'}) 31
frozenset({'Hershey', 'KitKat'}) 30
frozenset({'AlmondJoy', 'Hershey'}) 29
